In [5]:
import nltk
from nltk.corpus import movie_reviews
import random
import pandas as pd

nltk.download('movie_reviews')

documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.raw(fileid), category))

random.shuffle(documents)

data = pd.DataFrame(documents, columns=["review", "sentiment"])
data.head()


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


,review,sentiment
0,"while it was part of former yugoslavia , my co...",pos
1,""" desperate measures "" is a generic title for...",neg
2,i guess that if a very wild bachelor party had...,neg
3,notting hill's trailer is awful : a laughless ...,pos
4,"post-chasing amy , a slew of love-triangle mov...",neg
